In [3]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)


3.14162604


In [8]:
sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()

[[0], [2], [3], [4], [6]]

In [11]:
import os
import glob
import numpy as np
import cv2
import pymongo
import gridfs
from pymongo import MongoClient

def writeImageToDataBase(folder,collection,gfs):
    img_data = [img for img in glob.glob(os.path.join(folder,'*png'))]
    for idx,img_path in enumerate(img_data):
        print('IMAGE :',idx,img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_data_id = gfs.put(img.tobytes())
        name=os.path.basename(img_path)
        img_doc = {"name":name,'id':idx,'data_id':img_data_id,'shape':{'nx':img.shape[0],'ny':img.shape[1],'nz':img.shape[2]}}
        img_id = collection.insert_one(img_doc).inserted_id
        print('DOC : ',img_id)    

def getImageCollectionFromDataBase(collection,gfs):
    import pprint
    for img in collection.find():
        pprint.pprint(img)
        print('ID:',img['id'])
        shape=img['shape']
        print('SHAPE',shape['nx'],shape['ny'],shape['nz'])
        data_file = gfs.find_one({"_id":img['data_id']})
        bytes_data = data_file.read()
        print('NB BYTES',bytes_data.__len__(),np.dtype('uint8').itemsize)
        array_data = np.frombuffer(bytes_data,dtype='uint8',count=int(bytes_data.__len__()/np.dtype('uint8').itemsize))
        array3D_data =np.reshape(array_data,(shape['nx'],shape['ny'],shape['nz']))
        print('SHAPE2 :',array3D_data.shape)

In [12]:
client = MongoClient('mongodb://%s:%s@islin-hdpnod1.ifp.fr' % ("gratienj", "gratienj2019!"))

#
# GETTING a Database
db = client['tgs-img']
grid_fs = gridfs.GridFS(db,'bdata')

# GETTING a collection
img_collection = db['img-collection']
img=img_collection.find_one({"id":0})
print('ID:',img['id'])
shape=img['shape']
print('SHAPE',shape['nx'],shape['ny'],shape['nz'])
data_file = grid_fs.find_one({"_id":img['data_id']})
bytes_data = data_file.read()
print('NB BYTES',bytes_data.__len__(),np.dtype('uint8').itemsize)
array_data = np.frombuffer(bytes_data,dtype='uint8',count=int(bytes_data.__len__()/np.dtype('uint8').itemsize))
array3D_data =np.reshape(array_data,(shape['nx'],shape['ny'],shape['nz']))
print('SHAPE2 :',array3D_data.shape)

ID: 0
SHAPE 101 101 3
NB BYTES 30603 1
SHAPE2 : (101, 101, 3)


In [46]:
def countPartionSize(row_iterator):
    count = 0
    for row in row_iterator:
        count = count+1
    yield count
rdd = sc.parallelize(array3D_data)
print("NUM PARTITION : ",rdd.getNumPartitions())

NUM PARTITION :  32


In [47]:
print(rdd.first().shape)
nrdd=rdd.map(lambda x: (x.shape[0], x.shape[1])).collect()
print(nrdd)

(101, 3)
[(101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), (101, 3), 

In [48]:
rdd2=rdd.mapPartitions(countPartionSize).collect()
print(rdd2)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5]
